In [0]:
%run "/Workspace/Formula1Project/includes/configuration"

In [0]:
%run "/Workspace/Formula1Project/includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")
print(v_file_date)

In [0]:
circuits_df = spark.read.format("delta").load(f"{processed_path}/circuits", header=True,inferSchema=True).withColumnRenamed("name","circuits_name")
races_df = spark.read.format("delta").load(f"{processed_path}/races", header=True,inferSchema=True).withColumnRenamed("name","races_name")
drivers_df = spark.read.format("delta").load(f"{processed_path}/drivers", header=True,inferSchema=True).withColumnRenamed("name","drivers_name")
constructor_df = spark.read.format("delta").load(f"{processed_path}/constructors", header=True,inferSchema=True).withColumnRenamed("name","constructors_name")
results_df = spark.read.format("delta").load(f"{processed_path}/results", header=True,inferSchema=True)

In [0]:
display(results_df)

In [0]:
from pyspark.sql.functions import current_timestamp, lit, col
results_df = results_df.filter(f"file_date == '{v_file_date}'")

In [0]:
display(results_df)

In [0]:
rc_df = races_df.join(circuits_df, on='circuit_id', how="inner")

In [0]:
rc_results_df = rc_df.join(results_df, on='race_id', how="inner")

In [0]:
rc_results_driver_df = rc_results_df.join(drivers_df, on='driver_id', how="inner")

In [0]:
final_df = rc_results_driver_df.join(constructor_df, on='constructor_id', how="inner")['race_id','race_year','races_name','race_timestamp','location','drivers_name',drivers_df.number,drivers_df.nationality,'constructors_name','grid','fastest_lap','time','points','position',results_df.file_date]

In [0]:
from pyspark.sql.functions import current_timestamp
final_df = final_df.withColumnRenamed("race_timestamp","race_date").withColumnRenamed("location","circuit_location").withColumnRenamed("drivers_name","driver_name").withColumnRenamed("number","driver_number").withColumnRenamed("nationality","driver_nationality").withColumnRenamed("constructors_name","team").withColumnRenamed("time","race_time").withColumn("created_date",current_timestamp()).withColumnRenamed("file_date","results_file_date").orderBy("points", ascending=False)

In [0]:
display(final_df)

In [0]:
merge_condition = "tgt.race_id = src.race_id and tgt.driver_name = src.driver_name"
merge_data(final_df, presentation_path, "f1_presentation", "race_results", "race_id", merge_condition)

In [0]:
#overwrite_partition(final_df,'race_id', 'f1_presentation','race_results')

In [0]:
#final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.race_results")